In [1]:
import s3fs
import pandas as pd
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})
MY_BUCKET = "jhajjar"
FILE_PATH_S3 = f"{MY_BUCKET}/Diffusion/listings.csv"
with fs.open(FILE_PATH_S3, "r") as file_in:
    listings = pd.read_csv(file_in)

listings

,hektor-fmpconsults-474,Appartement,3,75001 Paris 1er (Vendôme),61,847 600 €
0,apimo-85473173,Appartement,1.0,75001 Paris 1er (Châtelet - Les Halles),29.0,364 000 €
1,hektor-Les-4-Quartiers-383,Appartement,1.0,75001 Paris 1er (Châtelet - Les Halles),23.0,301 000 €
2,hektor-PARISLUXURYHOMES-4007,Appartement,5.0,75002 Paris 2e (Palais Royal),106.0,2 490 000 €
3,ag754594-448898324,Studio,NaN,75001 Paris 1er (Saint-Germain - L'Auxerrois),27.0,365 000 €
4,apimo-85473173,Appartement,1.0,75001 Paris 1er (Châtelet - Les Halles),29.0,364 000 €
...,...,...,...,...,...,...
9762,apimo-6951400,Appartement,4.0,75020 Paris 20e (Plaine - Lagny),89.0,599 000 €
9763,apimo-6810785,Appartement,2.0,75020 Paris 20e (Gambetta),40.0,302 000 €
9764,immo-facile-49166437,Appartement,3.0,75020 Paris 20e (Père Lachaise - Réunion),67.0,769 000 €
9765,immo-facile-48762452,Appartement,3.0,75020 Paris 20e (Télégraphe - Pelleport - Sain...,57.0,519 000 €


Nettoyage + simplification

In [ ]:
listings.dropna(inplace=True)
nv_noms = {'hektor-fmpconsults-474' : 'ID', 'Appartement' : 'Type', '3' : 'Number_of_rooms', '75001 Paris 1er (Vendôme)' : 'localisation', '61' : 'sqm2','847 600 €' : 'price'}
listings.rename(columns= nv_noms, inplace=True)
listings = listings.drop(['ID','localisation'],axis=1)
listings.head()

,Type,Number_of_rooms,sqm2,price
0,Appartement,1.0,29.0,364 000 €
1,Appartement,1.0,23.0,301 000 €
2,Appartement,5.0,106.0,2 490 000 €
4,Appartement,1.0,29.0,364 000 €
5,Appartement,2.0,67.0,1 190 000 €


On format correctement le prix en enlevant les espaces et le "€". Avant cela nous devons enlever les lignes problématiques à savoir celles qui ne sont pas du format "xxx xxx €".

In [16]:
listings = listings[listings['price'].str.match(r'^\d+(\s?\d+)*\s€$', na=False)]
listings['price'] = listings['price'].str.replace(' ', '').str[:-1].astype(int)
listings['price'].dtype

/tmp/ipykernel_125033/2532443576.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings['price'] = listings['price'].str.replace(' ', '').str[:-1].astype(int)


dtype('int64')

Description des colonnes "number of rooms", "sqm2" et "price"

In [19]:
desc = listings[['Number_of_rooms', 'sqm2', 'price']].describe().round(2)
desc

,Number_of_rooms,sqm2,price
count,9358.00,9358.00,9358.00
mean,2.94,69.62,806740.49
std,1.83,53.07,877042.55
min,1.00,4.00,22000.00
25%,2.00,35.00,350000.00
50%,3.00,56.00,565000.00
75%,4.00,88.00,935000.00
max,90.00,699.00,20550000.00


Le sample est donc de 9358 observations ce qui nous permet de faire des interprétations sans se baser sur un échantillon qui manquerait de représentativité. Cependant la ville de Paris sur laquelle nous nous focalisons comporte en elle-même un biais par rapport aux autres villes françaises(centre économique, centre politique, centre culturel et dimension internationale)
La moyenne (806 740€) est sensiblement plus élevé que la médiane (565 000€) ce qui suppose que la distribution de prix est biaisé. Le prix de quelques propriétés tirent à la hausse la moyenne. Cela peut paraît logique, la ville de Paris étant connu pour son immobilier de luxe, prisée par des investisseurs internationaux. Le maximum de 20 millions d'euros confirme cette disparité dans les prix.